In [ ]:
!pip freeze | grep scikit-learn

In [1]:
import pickle
import pandas as pd
import pyarrow
import numpy

In [2]:
def load_model():
    with open('model.bin', 'rb') as f_in:
        dv, model = pickle.load(f_in)
    return dv, model

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()
    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df


In [13]:
year = 2022
month = 2

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{year:04d}-{month:02d}.parquet'


In [ ]:
df = read_data(input_file)

In [7]:
dv, model = load_model()
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

c:\Users\mouba\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mouba\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
numpy.std(y_pred)

5.28140357655334

In [14]:
def apply_model(input_file, output_file):

    df = read_data(input_file)

    dv, model = load_model()
    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    
    y_pred = model.predict(X_val)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred
    
    df_result.to_parquet(output_file, engine='pyarrow', compression=None, index=False)

In [15]:
apply_model(input_file, output_file)

c:\Users\mouba\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\mouba\anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
